# Vision 기능

- GPT-4o with image inpus

- GPT-4o와 GPT-4 Turbo 모두 비전 기능을 갖추고 있습니다. 즉, 모델이 이미지를 촬영하고 그에 대한 질문에 답할 수 있습니다

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

이미지는 두 가지 주요 방법, 즉 이미지에 대한 링크를 전달하거나 요청에 base64로 인코딩된 이미지를 직접 전달하는 방법으로 모델에서 사용할 수 있습니다. 

In [2]:
from openai import OpenAI

client = OpenAI()

### 이미지의 내용 설명
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg" width=300px />op)을 생성합니다.

In [10]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "이 이미지의 내용을 설명해줘. 한국어로 설명해줘."},
        {
          "type": "image_url",
          "image_url": {
            "url": url
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

response.to_dict()

{'id': 'chatcmpl-Auogb8h0S0f7Cj00vPB4D3xAEuFve',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '이 이미지는 넓은 초원과 나무들로 둘러싸인 보행로를 보여줍니다. 보행로는 나무로 만들어져 있으며, 초록색 풀밭 사이로 뻗어 있습니다. 하늘은 푸르고 구름이 흩어져 있어 맑은 날씨를 나타내고 있습니다. 전체적으로 평화롭고 자연적인 풍경이 느껴지는 장면입니다.',
    'refusal': None,
    'role': 'assistant'}}],
 'created': 1738104569,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_bd83329f63',
 'usage': {'completion_tokens': 95,
  'prompt_tokens': 1160,
  'total_tokens': 1255,
  'completion_tokens_details': {'audio_tokens': 0,
   'reasoning_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [11]:
print(response.choices[0].message.content)

이 이미지는 넓은 초원과 나무들로 둘러싸인 보행로를 보여줍니다. 보행로는 나무로 만들어져 있으며, 초록색 풀밭 사이로 뻗어 있습니다. 하늘은 푸르고 구름이 흩어져 있어 맑은 날씨를 나타내고 있습니다. 전체적으로 평화롭고 자연적인 풍경이 느껴지는 장면입니다.


### Base 64로 인코딩된 이미지 업로드
로컬에 이미지 또는 이미지 세트가 있는 경우 이를 Base 64 인코딩 형식으로 모델에 전달할 수 있습니다. 다음은 이에 대한 실제 예입니다.

In [7]:
import base64
import requests

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "data/냉장고내용물.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {openai.api_key }"
}

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "이 이미지의 내용을 한국어로 설명해줘."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ],
    }
  ],
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='이 이미지는 잘 정리된 냉장고의 내부를 보여줍니다. 냉장고에는 다양한 신선한 채소와 과일이 가득 차 있습니다. 상단 선반에는 여러 종류의 과일, 예를 들어 딸기, 레몬, 사과와 같은 것들이 놓여 있고, 중간 선반에는 다양한 채소가 배열되어 있습니다. 바닥 서랍에는 여러 가지 채소와 과일이 잘 정리되어 있으며, 음료수와 유제품도 다양한 색상의 용기에 담겨 있습니다. 전체적으로 매우 건강하고 신선한 식품들이 가득한 모습입니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))


### 다중 이미지 입력
Chat Completions API는 base64로 인코딩된 형식이나 이미지 URL로 여러 이미지 입력을 가져와 처리할 수 있습니다. 모델은 각 이미지를 처리하고 모든 이미지의 정보를 사용하여 질문에 답합니다.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg" width=300px />

<img src="https://img.freepik.com/premium-photo/a-wooden-walkway-leads-to-a-green-field-with-a-mountain-in-the-background_800563-10576.jpg" width=300px />

In [8]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "이 이미지에는 무엇이 들어있나요? 그들 사이에 어떤 차이가 있나요?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://img.freepik.com/premium-photo/a-wooden-walkway-leads-to-a-green-field-with-a-mountain-in-the-background_800563-10576.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

In [9]:
print(response.choices[0].message.content)

첫 번째 이미지는 맑은 하늘과 초록의 풀밭이 펼쳐진 경치로, 나무로 만들어진 산책로가 초원을 지나가는 모습입니다. 푸르른 식물과 맑은 날씨가 주는 평화로운 분위기가 특징적입니다.

두 번째 이미지는 지평선에 있는 산과 함께 석양이 드리워진 풍경으로, 길게 펼쳐진 길이 밀밭을 지나고 있는 모습입니다. 이 이미지에서는 시간대가 저녁이나 아침쯤으로 보이며, 빛과 그림자의 대비가 강조됩니다.

두 이미지의 주된 차이는 분위기와 시간대입니다. 첫 번째 이미지는 낮의 생동감 넘치는 느낌을 주는 반면, 두 번째 이미지는 석양의 차분한 느낌을 담고 있습니다. 또한, 첫 번째는 초원과 숲이 있는 반면, 두 번째는 밀밭과 산이 배경으로 되어 있습니다.
